In [1]:
import psycopg2

# Database 연결
host = 'drona.db.elephantsql.com'
user = 'rzdqhkrx'
password = 'ZXxLzVgHRopX8mgUqreVyxsWCq04LYmD'
database = 'rzdqhkrx'

connection = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# 데이터 로드
cursor = connection.cursor()

cursor.execute("""SELECT e.id, e."month", e.metro, e.cntr, e.custCnt, e.powerusage, e.bill, e.unitcost,
                w.pa, w.avgtamax, w.avgtamin, w.taavg, w.avghm, w.sumssday, w.daydur
                FROM electricity e
                JOIN weather w ON e."month" = w."month"
                ORDER BY e.id
                """)
data = cursor.fetchall()
type(data)

list

In [15]:
import pandas as pd

df = pd.DataFrame(data,
                  columns=['id', 'month', 'metro', 'cntr', 'custCnt', 'powerusage', 'bill', 'unitcost', 'pa', 'avgtamax', 'avgtamin', 'taavg', 'avghm', 'sumssday', 'daydur'])
'''
month : 조회월
metro : 시도명
cntr : 계약종별
custCnt : 전기사용 고객 호수(호)
powerUsage : 전력사용량 합계(kWh)
bill : 전기요금 합계(원)	
unitCost : 전기요금 평균판매 단가(원/kWh)
pa : 평균 현지기압(hPa)
avgtamax : 최고기온의 평균(℃)
avgtamin : 최저기온의 평균(℃) 
taavg : 평균 기온(℃)
avghm : 평균상대습도(%)
sumssday : 일조 총 시간(hr)
daydur : 일조-일조율(%)
'''

df

,id,month,metro,cntr,custCnt,powerusage,bill,unitcost,pa,avgtamax,avgtamin,taavg,avghm,sumssday,daydur
0,22,2020-04-01,세종,주택용,53367,44849380,4830177162,107.7,1006.5,17.9,4.3,10.8,48,279.6,71
1,23,2020-04-01,세종,일반용,13511,51109127,6226001078,121.8,1006.5,17.9,4.3,10.8,48,279.6,71
2,24,2020-04-01,세종,교육용,143,4556805,424085745,93.1,1006.5,17.9,4.3,10.8,48,279.6,71
3,25,2020-04-01,세종,산업용,1636,147549374,13364118435,90.6,1006.5,17.9,4.3,10.8,48,279.6,71
4,26,2020-04-01,세종,농사용,9558,6121652,304288701,49.7,1006.5,17.9,4.3,10.8,48,279.6,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,248,2022-12-01,세종,교육용,157,9115595,1081253177,118.6,1014.0,2.6,-6.7,-2.4,64,153.5,51
227,249,2022-12-01,세종,산업용,1731,176017026,25067227514,142.4,1014.0,2.6,-6.7,-2.4,64,153.5,51
228,250,2022-12-01,세종,농사용,10603,6996828,487571459,69.7,1014.0,2.6,-6.7,-2.4,64,153.5,51
229,251,2022-12-01,세종,가로등,14705,3764228,457902405,121.6,1014.0,2.6,-6.7,-2.4,64,153.5,51


In [16]:
df['month'] = pd.to_datetime(df['month']).dt.strftime('%m')
df['cntr'] = df['cntr'].replace('주택용',0).replace('일반용',1).replace('교육용',2).replace('산업용',3).replace('농사용',4).replace('가로등',5).replace('심 야',6)
df['cntr'] = df['cntr'].astype('category')
df = df.drop(['id', 'metro'], axis=1)
df
# 세종의 2020-04 이전 날씨 정보가 없음.

,month,cntr,custCnt,powerusage,bill,unitcost,pa,avgtamax,avgtamin,taavg,avghm,sumssday,daydur
0,04,0,53367,44849380,4830177162,107.7,1006.5,17.9,4.3,10.8,48,279.6,71
1,04,1,13511,51109127,6226001078,121.8,1006.5,17.9,4.3,10.8,48,279.6,71
2,04,2,143,4556805,424085745,93.1,1006.5,17.9,4.3,10.8,48,279.6,71
3,04,3,1636,147549374,13364118435,90.6,1006.5,17.9,4.3,10.8,48,279.6,71
4,04,4,9558,6121652,304288701,49.7,1006.5,17.9,4.3,10.8,48,279.6,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,12,2,157,9115595,1081253177,118.6,1014.0,2.6,-6.7,-2.4,64,153.5,51
227,12,3,1731,176017026,25067227514,142.4,1014.0,2.6,-6.7,-2.4,64,153.5,51
228,12,4,10603,6996828,487571459,69.7,1014.0,2.6,-6.7,-2.4,64,153.5,51
229,12,5,14705,3764228,457902405,121.6,1014.0,2.6,-6.7,-2.4,64,153.5,51


In [27]:
X = df.drop(['bill','unitcost','powerusage'], axis=1)
y = df['bill']

In [28]:
X

,month,cntr,custCnt,pa,avgtamax,avgtamin,taavg,avghm,sumssday,daydur
0,04,0,53367,1006.5,17.9,4.3,10.8,48,279.6,71
1,04,1,13511,1006.5,17.9,4.3,10.8,48,279.6,71
2,04,2,143,1006.5,17.9,4.3,10.8,48,279.6,71
3,04,3,1636,1006.5,17.9,4.3,10.8,48,279.6,71
4,04,4,9558,1006.5,17.9,4.3,10.8,48,279.6,71
...,...,...,...,...,...,...,...,...,...,...
226,12,2,157,1014.0,2.6,-6.7,-2.4,64,153.5,51
227,12,3,1731,1014.0,2.6,-6.7,-2.4,64,153.5,51
228,12,4,10603,1014.0,2.6,-6.7,-2.4,64,153.5,51
229,12,5,14705,1014.0,2.6,-6.7,-2.4,64,153.5,51


In [29]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor()
regr.fit(X, y)

RandomForestRegressor()

In [30]:
regr.score(X,y)

0.9391977555948009

In [31]:
print(int(regr.predict([['04', '0',	'53367', '1006.5',	'17.9',	'4.5', '10.8', '48', '279.6', '70']])))

4874084033


c:\Users\Juyeon\anaconda3\envs\project4\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [32]:
from joblib import dump
dump(regr, 'model.pkl') 

['model.pkl']